In [4]:
import pymysql
from configparser import ConfigParser

config = ConfigParser()
config.read('../Chapter1/config.ini') # 指定設定檔的檔案路徑

connection = pymysql.connect(
    host=config.get('DB', 'host'),
    user=config.get('DB', 'user'),
    password=config.get('DB', 'password'),
    port=config.getint('DB', 'port'),
    cursorclass=pymysql.cursors.DictCursor,
)

print(connection.open)

True


建立資料庫

In [6]:
from pprint import pprint
with connection.cursor() as cursor:
    sql = """
    CREATE DATABASE IF NOT EXISTS chapter2;
    """
    # 執行寫入的 SQL 語句
    cursor.execute(sql)
    
    # 執行查詢資料表
    

    sql = """SHOW DATABASES;"""
    cursor.execute(sql)
    # 取得查詢的所有資料
    r = cursor.fetchall()
print(r)


[{'Database': 'chapter2'}, {'Database': 'classicmodels'}, {'Database': 'imdb_top250_database'}, {'Database': 'information_schema'}, {'Database': 'my_database'}, {'Database': 'my_titanic'}, {'Database': 'my_train_titanic'}, {'Database': 'mysql'}, {'Database': 'performance_schema'}, {'Database': 'sakila'}, {'Database': 'social_media_app'}, {'Database': 'sys'}, {'Database': 'transaction_test'}, {'Database': 'world'}]


建立資料表

In [13]:
connection = pymysql.connect(
    host=config.get('DB', 'host'),
    user=config.get('DB', 'user'),
    password=config.get('DB', 'password'),
    port=config.getint('DB', 'port'),
    database='chapter2',
    cursorclass=pymysql.cursors.DictCursor,
)


with connection.cursor() as cursor:
    """ 建立 user 資料表
        id 主鍵
        name 字串 不能為空
        age 整數 
        username 字串 不能為空 必須唯一
        password 字串 不能為空
    """
    sql = """

    CREATE TABLE IF NOT EXISTS users(
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    age INT,
    username VARCHAR(255) NOT NULL UNIQUE,
    password VARCHAR(255) NOT NULL
    )


    """
    # 執行建立的 SQL 語句
    cursor.execute(sql)
    # 執行查看資料庫
    cursor.execute("SHOW TABLES FROM chapter2;")
    # 取得查詢的所有資料
    r = cursor.fetchall()
    print(r)




[{'Tables_in_chapter2': 'users'}]


寫入資料

提交資料庫變更 `conncection.commit()`

In [ ]:
from pprint import pprint
with connection.cursor() as cursor:
    sql = """
   INSERT INTO users (name, age, username, password)
   VALUES("jarvis",25,"jarvis","123456")
    """
    # 執行寫入的 SQL 語句
    cursor.execute(sql)
    
    # 執行查詢資料表
    connection.commit() #要commit才會寫入資料庫

    cursor.execute("SELECT * FROM users;")
    # 取得查詢的所有資料
    r = cursor.fetchall()
pprint(r)


[{'age': 25,
  'id': 1,
  'name': 'jarvis',
  'password': '123456',
  'username': 'jarvis'}]


In [17]:
print(cursor.rowcount) #影響的行數

1


使用另一個連線查詢資料庫

In [16]:
connection2 = pymysql.connect(
    host=config.get('DB', 'host'),
    user=config.get('DB', 'user'),
    password=config.get('DB', 'password'),
    port=config.getint('DB', 'port'),
    cursorclass=pymysql.cursors.DictCursor,
    db='chapter2',
)
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM users;")
    result = cursor.fetchall()

pprint(result)

[{'age': 25,
  'id': 1,
  'name': 'jarvis',
  'password': '123456',
  'username': 'jarvis'}]


In [18]:
from pprint import pprint
with connection.cursor() as cursor:
    sql = """
    INSERT INTO users (name, age, username, password)
    VALUES("Alex",28,"alex123","56789")
    """
    cursor.execute(sql)
    # 提交資料庫的變更
    connection.commit()
    cursor.execute("SELECT * FROM users;")
    # 取得查詢的所有資料
    r = cursor.fetchall()
print(r)
    
    

[{'id': 1, 'name': 'jarvis', 'age': 25, 'username': 'jarvis', 'password': '123456'}, {'id': 2, 'name': 'Alex', 'age': 28, 'username': 'alex123', 'password': '56789'}]


更新資料

In [ ]:
with connection.cursor() as cursor:
    sql = """
    INSERT INTO users (name, age, username, password)
    VALUES("Alex",28,"alex123","56789");
    """
    cursor.execute(sql)
    
    # 提交資料庫的變更
    connection.commit()
    
    cursor.execute("SELECT * FROM users;")
    result = cursor.fetchall()
pprint(result)

[{'age': 26,
  'id': 1,
  'name': 'Jarvis Lu',
  'password': '123456',
  'username': 'jarvis'},
 {'age': 28,
  'id': 2,
  'name': 'Alex',
  'password': '56789',
  'username': 'alex123'}]


刪除資料表

In [ ]:
try:
    with connection.cursor() as cursor:
        sql = """
        DELETE FROM users WHERE name = 'Jarvis';
        """
        cursor.execute(sql)
        print(f"影響資料筆數:{cursor.rowcount}筆")
        # if cursor.rowcount == 1 :
        #     connection.rollback()
        
        cursor.execute("SELECT * FROM users;")
        result = cursor.fetchall()
        
except:
    print("發生錯誤 取消更動")
    connection.rollback()
finally:
    connection.commit()
print(result)

發生錯誤 取消更動
[{'id': 1, 'name': 'Jarvis Lu', 'age': 26, 'username': 'jarvis', 'password': '123456'}]


In [28]:
try:
    with connection.cursor() as cursor:
        sql = """
        DELETE FROM users WHERE name = 'Alex';
        """
        cursor.execute(sql)
        print(f"影響資料筆數:{cursor.rowcount}筆")
        # if cursor.rowcount == 1 :
        #     connection.rollback()
        
        cursor.execute("SELECT * FROM users;")
        result = cursor.fetchall()
        
except:
    print("發生錯誤 取消更動")
    connection.rollback()
finally:
    connection.commit()
print(result)

影響資料筆數:1筆
[{'id': 1, 'name': 'Jarvis Lu', 'age': 26, 'username': 'jarvis', 'password': '123456'}]


In [27]:
try:
    with connection.cursor() as cursor:
        sql = """
        INSERT INTO users (name, age, username, password)
        VALUES("Alex",28,"alex123","56789");
        """
        cursor.execute(sql)
        print(f"影響資料筆數:{cursor.rowcount}筆")
        # if cursor.rowcount == 1 :
        #     connection.rollback()
        
        cursor.execute("SELECT * FROM users;")
        result = cursor.fetchall()
        
except:
    print("發生錯誤 取消更動")
    connection.rollback()
finally:
    connection.commit()
print(result)

影響資料筆數:1筆
[{'id': 1, 'name': 'Jarvis Lu', 'age': 26, 'username': 'jarvis', 'password': '123456'}, {'id': 3, 'name': 'Alex', 'age': 28, 'username': 'alex123', 'password': '56789'}]


In [2]:
from sql_utils import MySQLManager

mysql = MySQLManager('chapter2')
tables=mysql.sql_query("SHOW TABLES FROM chapter2")
print(tables)
user = mysql.get_user_info('jarvis','123456')
print(user)

[{'Tables_in_chapter2': 'users'}]
[{'id': 1, 'name': 'Jarvis Lu', 'age': 26, 'username': 'jarvis', 'password': '123456'}]
